## Import necessary modules
Run this cell before running any other cells

In [1]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

# Printing and Logging
## Printing
You can use the **print()** function in Python to print messages to the screen. <br>
The message can be a string, or any other object, the object will be converted into a string before it is written to the screen. <br>

## Logging
You could use the logging module that is setup in *utils.py*. <br>
It prints to both your screen (standard output) as well as to log files (*ble.log*) in the *logs* directory. <br>
This is the recommended way to output messages, since the log files can help with debugging. <br>
The logging module also provides different log levels as shown below, each formatted with a different color for increased visibility. <br>

__**NOTE**__: You may notice that the DEBUG message is not printed to the screen but is printed in the log file. This is because the logging level for the screen is set to INFO and for the file is set to DEBUG. You can change the default log levels in *utils.py* (**STREAM_LOG_LEVEL** and **FILE_LOG_LEVEL**). 

## Formatting output
To format your strings, you may use %-formatting, str.format() or f-strings. <br>
The most "pythonic" way would be to use f-strings. [Here](https://realpython.com/python-f-strings/) is a good tutorial on f-strings. <br>

In [2]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

2022-02-03 18:23:12,393 | INFO     |: info
2022-02-03 18:23:12,395 | WARNING  |: warning
2022-02-03 18:23:12,401 | ERROR    |: error
2022-02-03 18:23:12,402 | CRITICAL |: critical


<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [26]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2022-02-03 21:25:15,553 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:83:B0:6C:9A:3C
2022-02-03 21:25:18,102 | INFO     |: Connected to C0:83:B0:6C:9A:3C


## Receive data from the Artemis board

The cell below shows examples of reading different types (as defined in the Arduino sketch) of GATT characteristics.

In [6]:
# Read a float GATT Charactersistic
f = ble.receive_float(ble.uuid['RX_FLOAT'])
print(f)

10.5


In [7]:
# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

[->9.0<-]


## Send a command to the Artemis board
Send the PING command and read the reply string from the string characteristic RX_STRING. <br>
__NOTE__: The **send_command()** essentially sends a string data to the GATT characteristic (TX_CMD_STRING). The GATT characteristic in the Arduino sketch is of type BLECStringCharacteristic.

In [8]:
ble.send_command(CMD.PING, "")

In [9]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

PONG


The cell below shows an example of the SEND_TWO_INTS command. <br> The two values in the **data** are separated by a delimiter "|". <br>
Refer Lab 2 documentation for more information on the command protocol.

In [10]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

The Artemis board should print the two integers to the serial monitor in the ArduinoIDE. 

In [11]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2022-02-03 18:24:57,872 | INFO     |: Already connected to a BLE device


In [12]:
ble.send_command(CMD.ECHO, "HiHello")

s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

Robot says -> HiHello :)


In [13]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2022-02-03 18:25:04,848 | INFO     |: Already connected to a BLE device


In [14]:
ble.send_command(CMD.SEND_THREE_FLOATS, "1.4|2.5|-1.7")

In [34]:
received_float = 0.0
received_times = []
start_time = time.time()

def receive_float_notif_handler(uuid, byte_array_value):
    received_time = time.time()
    global received_float, start_time
    received_float = ble.bytearray_to_float(byte_array_value)
    print(received_float)
    received_times.append(received_time-start_time)
    start_time = time.time()
    return received_float

ble.start_notify(ble.uuid['RX_FLOAT'], receive_float_notif_handler)
ble.send_command(CMD.SEND_THREE_FLOATS, "1.4|2.5|-1.7")

while (len(received_times) < 100):
    time.sleep(1)

ble.stop_notify(ble.uuid['RX_FLOAT'])

403.0


Exception ignored in: <function CentralManagerDelegate.__del__ at 0x7f8e8f465040>
Traceback (most recent call last):
  File "/Users/roninsharma/Desktop/Cornell/ECE 5960/ece5960_ble/lib/python3.9/site-packages/bleak/backends/corebluetooth/CentralManagerDelegate.py", line 102, in __del__
IndexError: NSRangeException - Cannot remove an observer <CentralManagerDelegate 0x7f8e8b3caab0> for the key path "isScanning" from <CBCentralManager 0x6000018bd080> because it is not registered as an observer.


ValueError: Characteristic notifications already started

403.5
404.0
404.5
405.0
405.5
406.0
406.5
407.0
407.5
408.0
408.5
409.0
409.5
410.0
410.5
411.0
411.5
412.0
412.5
413.0
413.5
414.0
414.5
415.0
415.5
416.0
416.5
417.0
417.5
418.0
418.5
419.0
419.5
420.0
420.5
421.0
421.5
422.0
422.5
423.0
423.5
424.0
424.5
425.0
425.5
426.0
426.5
427.0
427.5
428.0
428.5
429.0
429.5
430.0
430.5
431.0
431.5
432.0
432.5
433.0
433.5
434.0
434.5
435.0
435.5
436.0
436.5
437.0
437.5
438.0
438.5
439.0
439.5
440.0
440.5
441.0
441.5
442.0
442.5
443.0
443.5
444.0
444.5
445.0
445.5
446.0
446.5
447.0
447.5
448.0
448.5
449.0
449.5
450.0
450.5
451.0
451.5
452.0
452.5
453.0
453.5
454.0
454.5
455.0
455.5
456.0
456.5
457.0
457.5
458.0
458.5
459.0
459.5
460.0
460.5
461.0
461.5
462.0
462.5
463.0
463.5
464.0
464.5
465.0
465.5
466.0
466.5
467.0
467.5
468.0
468.5
469.0
469.5
470.0
470.5
471.0
471.5
472.0
472.5
473.0
473.5
474.0
474.5
475.0
475.5
476.0
476.5
477.0
477.5
478.0
478.5
479.0
479.5
480.0
480.5
481.0
481.5
482.0
482.5
483.0
483.5
484.0
484.5
485.0
485.5
486.0
486.

In [35]:
ble.stop_notify(ble.uuid['RX_FLOAT'])

In [37]:
np.mean(received_times)

0.5002288818359375

In [27]:
send_times = []

for i in range(1000):
    start_time = time.time()
    ble.send_command(CMD.SEND_TWO_INTS, "1|2")
    send_times.append(time.time()-start_time)

In [31]:
np.mean(send_times)

0.030822369813919066

## Disconnect

In [38]:
# Disconnect
ble.disconnect()

2022-02-03 21:45:16,999 | INFO     |: Disconnected from 607452C2-8441-D076-814E-841EB3635426
